In [1]:
import pandas as pd
import bjorn_support as bs
import mutations as bm
from path import Path
from Bio import SeqIO

In [28]:
out_fp = Path('/home/al/analysis/gorillas/all_seqs.fasta')
gor_seqs = Path('/home/al/analysis/gorillas/2021-01-13.fasta')
gisaid_seqs = Path('/home/al/analysis/gisaid/sequences_2021-01-13_08-22.fasta')
meta_fp = Path('/home/al/analysis/gisaid/metadata_2021-01-13_09-12.tsv.gz')

In [29]:
meta = pd.read_csv(meta_fp, sep='\t', compression='gzip')
meta = meta[~meta['location'].isna()]
meta = meta.loc[(meta['location'].str.contains('San Diego')) & (meta['country']=='USA')]

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [34]:
meta.loc[meta['strain'].str.contains('3713')]

,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,region_exposure,...,pangolin_lineage,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing
268961,USA/CA-ALSR-3713/2020,ncov,EPI_ISL_636160,?,2020-09-28,North America,USA,California,San Diego,North America,...,B.1.232,G,San Diego County Public Health Laboratory,Andersen lab at Scripps Research,SEARCH Alliance San Diego with Tracy Basler et al,https://www.gisaid.org,?,?,2020-11-13,?


In [4]:
sois = meta['strain'].unique().tolist()
len(sois)

2322

In [5]:
# bs.fetch_seqs(gisaid_seqs, out_fp, sois)

In [6]:
msa_fp = Path('/home/al/analysis/gorillas/all_seqs_aligned.fasta')


In [8]:
g_seqs = SeqIO.parse('/home/al/analysis/gorillas/2021-01-13.fasta', 'fasta')

In [9]:
g_ids = [rec.id for rec in g_seqs]

In [10]:
g_ids

['sample/ARTIC/hsapiends', 'sample/ARTIC/medaka/ggorilla']

In [15]:
subs = bm.identify_replacements(msa_fp, None)

Creating a dataframe...
Loading Alignment file at: /home/al/analysis/gorillas/all_seqs_aligned.fasta
Initial cleaning...
Identifying mutations...
Mapping Genes to mutations...
Compute codon numbers...
Fetch reference codon...
Fetch alternative codon...
Map amino acids...
Fuse with metadata...


In [16]:
def check_sample(x, sample_name):
    if sample_name in x.split(','):
        return True
    return False

In [17]:
subs['gorilla_mutation'] = False
subs['gorilla_mutation'] = subs['samples'].apply(check_sample, args=('sample/ARTIC/medaka/ggorilla',))

In [18]:
subs['hsapien_mutation'] = False
subs['hsapien_mutation'] = subs['samples'].apply(check_sample, args=('sample/ARTIC/hsapiends',))

In [27]:
subs[(subs['gorilla_mutation']==True) | (subs['hsapien_mutation']==True)].shape#.to_csv('/home/al/analysis/gorillas/gorilla_substitutions.csv', index=False)

(21, 11)

In [11]:
dels = bm.identify_deletions(msa_fp, None)

In [12]:
dels['gorilla_mutation'] = False
dels['gorilla_mutation'] = dels['samples'].apply(check_sample, args=('sample/ARTIC/medaka/ggorilla',))

In [13]:
dels['hsapien_mutation'] = False
dels['hsapien_mutation'] = dels['samples'].apply(check_sample, args=('sample/ARTIC/hsapiends',))

In [14]:
dels[(dels['gorilla_mutation']==True) | (dels['hsapien_mutation']==True)].to_csv('/home/al/analysis/gorillas/gorilla_deletions.csv', index=False)

,relative_coords,del_len,samples,num_samples,type,absolute_coords,pos,gene,codon_num,ref_codon,ref_aa,prev_5nts,next_5nts,gorilla_mutation,hsapien_mutation


In [21]:
subs.sort_values('num_samples', ascending=False)

,gene,ref_codon,alt_codon,pos,ref_aa,codon_num,alt_aa,num_samples,samples,gorilla_mutation,hsapien_mutation
3313,S,GAT,GGT,23403,D,614,G,2283,"USA/CA-PC101P/2020,USA/CA-MG0987/2020,USA/CA-S...",True,True
1510,ORF1ab,CTA,TTA,14408,L,4715,L,2283,"USA/CA-PC101P/2020,USA/CA-MG0987/2020,USA/CA-S...",True,True
2593,ORF1ab,TTC,TTT,3037,F,924,F,2275,"USA/CA-PC101P/2020,USA/CA-MG0987/2020,USA/CA-S...",True,True
87,5UTR,CGT,TGT,241,R,81,C,2266,"USA/CA-PC101P/2020,USA/CA-MG0987/2020,USA/CA-S...",True,True
2319,ORF1ab,TAG,TGG,20268,_,6668,W,1322,"USA/CA-SR0176/2020,USA/CA-SR0236/2020,USA/CA-S...",False,True
...,...,...,...,...,...,...,...,...,...,...,...
2181,ORF1ab,GTG,TTG,368,V,35,L,1,USA/CA-ALSR-1302/2020,False,False
984,ORF1ab,AGT,ATT,15672,S,5136,I,1,USA/CA-UW-1296/2020,False,False
985,ORF1ab,AGT,ATT,16197,S,5311,I,1,USA/CA-ALSR-1075-SAN/2020,False,False
2176,ORF1ab,GTG,GTT,13858,V,4531,V,1,USA/CA-ALSR-5089/2020,False,False


In [23]:
subs.loc[(subs['gorilla_mutation']==True) & (subs['hsapien_mutation']==True)].reset_index()

,index,gene,ref_codon,alt_codon,pos,ref_aa,codon_num,alt_aa,num_samples,samples,gorilla_mutation,hsapien_mutation
0,41,3UTR,GTA,TTA,29825,V,51,L,40,"USA/CA-ALSR-1866/2020,USA/CA-ALSR-3324/2020,US...",True,True
1,87,5UTR,CGT,TGT,241,R,81,C,2266,"USA/CA-PC101P/2020,USA/CA-MG0987/2020,USA/CA-S...",True,True
2,150,E,TTG,TTA,26325,L,27,L,61,"USA/CA-ALSR-2801/2020,USA/CA-ALSR-2874/2020,US...",True,True
3,325,N,CCC,CCT,28291,P,6,P,62,"USA/CA-ALSR-2801/2020,USA/CA-ALSR-2874/2020,US...",True,True
4,400,N,GAT,TAT,29474,D,401,Y,7,"USA/CA-ALSR-3324/2020,USA/CA-ALSR-3650/2020,US...",True,True
5,469,N,TAT,TAC,28600,Y,109,Y,5,"USA/CA-ALSR-3650/2020,USA/CA-ALSR-3713/2020,US...",True,True
6,475,N,TCA,TTA,28854,S,194,L,1307,"USA/CA-SR0176/2020,USA/CA-SR0236/2020,USA/CA-S...",True,True
7,559,ORF1ab,AAC,AAT,6706,N,2147,N,61,"USA/CA-ALSR-2801/2020,USA/CA-ALSR-2874/2020,US...",True,True
8,804,ORF1ab,ACC,ATC,6285,T,2007,I,62,"USA/CA-ALSR-2801/2020,USA/CA-ALSR-2874/2020,US...",True,True
9,854,ORF1ab,ACT,ATT,813,T,183,I,2,"sample/ARTIC/hsapiends,sample/ARTIC/medaka/ggo...",True,True


In [25]:
subs.loc[(subs['gorilla_mutation']==False) & (subs['hsapien_mutation']==True)].reset_index()

,index,gene,ref_codon,alt_codon,pos,ref_aa,codon_num,alt_aa,num_samples,samples,gorilla_mutation,hsapien_mutation
0,2319,ORF1ab,TAG,TGG,20268,_,6668,W,1322,"USA/CA-SR0176/2020,USA/CA-SR0236/2020,USA/CA-S...",False,True


In [26]:
subs.shape

(3521, 11)